## ВМА. Лабораторная 6. Отчет
### Гамезо Валерия

### Постановка задачи

Разработать программу вычисления наибольшего и второго по величине модуля собственных значений и соответствующих им собственных векторов симметричной матрицы. 

### Расчетные формулы

Рассчет $\lambda_1$ для несимметричной матрицы: 

$$\lambda_1 \approx \nu_i^{k+1} \; \text{sign} \; u_i^k, \text{где} \; i = \overline{1, n}, \nu_i^k = \| \nu^k \|_{\infty} $$

Для симметричной матрицы: 

$$\lambda_1 \approx \frac{(\nu^{k + 1}, u^k)}{(u^k, u^k)}$$

Рассчет вектора "погрешности" для $\lambda_1$:

$$ \nu^{k+1} – \lambda_1 u^k  $$

Рассчет $\lambda_2$:

$$ \lambda_2 \approx \frac{\nu^{m + 1}_i \| \nu^m \| - \lambda_1 \nu_i^m}{\nu_i^m - \lambda_1 u_i^{m - 1}}, \text{где} \; i = \overline{1, n}, \nu_i^m - \lambda_1 u_i^{m - 1} = \| \nu^m - \lambda_1 u^{m - 1} \|_{\infty}$$

Рассчет собственного вектора для $\lambda_2$:

$$ \nu^{k + 1} - \lambda_1 u^k $$

Рассчет вектора "погрешности" для $\lambda_2$:

$$ A x – \lambda_2 x  $$

### Входные данные

$$
A = \left( \begin{array}{rrrrrrrrrrrr}
  25 &  -3 &  -2 &  -2 &   0 &  -4 &  -2 &  -1 &  -2 &  -3 &  -2 &  -3 & \\
  -3 &  23 &  -2 &  -3 &  -1 &  -3 &  -4 &  -1 &  -1 &   0 &  -2 &  -3 & \\
  -2 &  -2 &  23 &  -2 &  -1 &  -2 &  -4 &   0 &  -4 &  -4 &  -1 &  -1 & \\
  -2 &  -3 &  -2 &  26 &   0 &  -3 &  -2 &  -3 &  -4 &  -2 &  -1 &  -4 & \\
   0 &  -1 &  -1 &   0 &  20 &  -4 &   0 &  -2 &  -3 &  -3 &  -4 &  -2 & \\
  -4 &  -3 &  -2 &  -3 &  -4 &  27 &   0 &  -1 &  -4 &  -4 &  -2 &   0 & \\
  -2 &  -4 &  -4 &  -2 &   0 &   0 &  18 &  -1 &   0 &  -1 &  -1 &  -3 & \\
  -1 &  -1 &   0 &  -3 &  -2 &  -1 &  -1 &  14 &  -2 &  -1 &   0 &  -2 & \\
  -2 &  -1 &  -4 &  -4 &  -3 &  -4 &   0 &  -2 &  28 &  -3 &  -1 &  -4 & \\
  -3 &   0 &  -4 &  -2 &  -3 &  -4 &  -1 &  -1 &  -3 &  25 &  -4 &   0 & \\
  -2 &  -2 &  -1 &  -1 &  -4 &  -2 &  -1 &   0 &  -1 &  -4 &  20 &  -2 & \\
  -3 &  -3 &  -1 &  -4 &  -2 &   0 &  -3 &  -2 &  -4 &   0 &  -2 &  24 &
\end{array} \right)
$$

### Листинг программы

Класс матрицы

```c++
class Matrix {
public:
    Matrix(const Matrix& matrix) : mSize(matrix.mSize) { //конструктор копирования
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];

            for (int j = 0; j < mSize; ++j) {
                a[i][j] = matrix[i][j];
            }
        }
    }
    
    Matrix(int size) : mSize(size) {
        a = new float* [mSize];

        for (int i = 0; i < mSize; ++i) {
            a[i] = new float[mSize];
        }

        generateCoefs();
    }

    ~Matrix() {
        for (int i = 0; i < mSize; ++i) {
            delete[] a[i];
        }
    }


    float* operator[] (int index) const {
        return a[index];
    }
    
    int size() const {
        return mSize;
    }
    
    Matrix& operator= (const Matrix& other) {
        if (this == &other) return *this;

        for (int i = 0; i < mSize; ++i) {
            for (int j = 0; j < mSize; ++j) {
                a[i][j] = other[i][j];
            }
        }

        return *this;
    }

private:
    void generateCoefs (bool isNull) { // генерация коеффициентов симметричной матрицы
    srand(time(nullptr));

    for (int i = 0; i < mSize; ++i) {
        for (int j = i + 1; j < mSize; ++j) {
            a[i][j] = std::rand() % 5 - 4;
            a[j][i] = a[i][j];
        }
    }
    
    for (int i = 0; i < mSize; ++i) {
        for (int j = 0; j < mSize; ++j) {
            if (i != j) {
                a[i][i] -= a[i][j];
            }
        }
    }
    
    a[0][0] ++;
    };
```

Класс, выполняющий итерации степенного метода и вычисляющий собственные вектора двух наибольших по модулю собственных значений заданной матрицы. Файл PowerMethod.h

```c++
    
class PowerMethod {
    public:
        explicit PowerMethod(const Matrix&);
        
        std::vector<float> getV();
        std::vector<float> getU();

        void calcNextIt();

        float getFirstLambdaCurEigenVal(bool); // вычисляет наибольшее по модулю собственное значение lambda_1
                                            // для симместричной и нессиметричной матриц, 
                                            // в зависимости от установленного логического флага
        std::vector<float> getFirstLambdaErrorVector(); // вычисляет значение вектора погрешности для 
                                                        // lambda_1 на текущей итерации k
    
        float getSecondLambdaEigenVal(std::vector<float>, std::vector<float>, std::vector<float>, float); // вычисляет
                                            // второе наибольшее по модулю собственное значение lambda_2
                                            // при заданных векторах v^{m + 1}, v^m, u^{m - 1} 
                                            // и значении lambda_1
        std::vector<float> getSecondLambdaEigenVector(std::vector<float>, std::vector<float>, float); // вычисляет
                                            // собственный вектор для lambda_2
                                            // при заданных векторах v^{m + 1}, u^m 
                                            // и значении lambda_1
        std::vector<float> getSecondLambdaErrorVector(std::vector<float>, float); // вычисляет значение 
                                                                    // вектора погрешности (собственный вектор x 
                                                                    // и собственное значение lambda_2 
                                                                    // заданы аргументами)

    private:
        Matrix matrix;
        std::vector<float> u; // нормированный вектор u_k при k, равном значении curI
        std::vector<float> v; // вектор v_{k + 1} при k, равном значении curI
        int curI;
};
```

PowerMethod.cpp

```c++
PowerMethod::PowerMethod(const Matrix& otherMatrix) // инициализация начального состояния
                                                    // u = u_0, k = 0
    : matrix(otherMatrix), curI(-1) {
  int size = matrix.size();

  u.resize(size);
  v.resize(size);
  v[0] = 1;

  calcNextIt();
}

std::vector<float> PowerMethod::getV() {
  return v;
}

std::vector<float> PowerMethod::getU() {
  return u;
}

void PowerMethod::calcNextIt() { // вычисление следующей итерации
  int size = matrix.size();

  float maxV = v[0]; // нормировка v^k для получения u^k
  for (int i = 0; i < size; ++i) {
    if (fabs(v[i]) > fabs(maxV)) {
      maxV = fabs(v[i]);
    }

    u[i] = v[i];
  } 

  for (int i = 0; i < size; ++i) {
    u[i] /= maxV;
  }

  for (int i = 0; i < size; ++i) { // вычисление v^{k + 1}
    v[i] = 0;

    for (int j = 0; j < size; ++j) {
      v[i] += matrix[i][j] * u[j];
    }
  }

  curI++; // инкремент счетчика
}

float PowerMethod::getFirstLambdaCurEigenVal(bool isSymmetric) {
  float lambda;
  int size = matrix.size();

  if (isSymmetric) { // вычисление lambda_1 для симметричной матрицы
    float num, den;
    num = den = 0;

    for (int i = 0; i < size; ++i) {
      num += v[i] * u[i]; // вычисление векторного произведения (v^{k + 1}, u^k)
      den += u[i] * u[i]; // вычисление векторного произведения (u^k, u^k)
    }

    lambda = num / den;
  } else { // вычисление lambda_1 для несимметричной матрицы
    int maxUIndex = 0;

    for (int i = 1; i < size; ++i) { // нахождение максиальной нормы вектора v^{k + 1}
      if (fabs(u[i]) > fabs(u[maxUIndex])) {
        maxUIndex = i;
      }
    }

    lambda = u[maxUIndex] < 0 ? -v[maxUIndex] : v[maxUIndex]; // умножение lambda_1 на sign (u^k_i)
  }

  return lambda;
}

std::vector<float> PowerMethod::getFirstLambdaErrorVector() {
  int size = matrix.size();
  float lambda = getFirstLambdaCurEigenVal(true);

  std::vector<float> error(size);

  for (int i = 0; i < size; ++i) {
    error[i] = v[i] - lambda * u[i]; // вычисление вектора v^{k + 1} - lambda_1 * u^k
  }

  return error;
}

float PowerMethod::getSecondLambdaEigenVal(std::vector<float> u_mMinus,
                                           std::vector<float> v_m,
                                           std::vector<float> v_mPlus,
                                           float lambda1) {
  int maxAbsDenIndex = 0;
  int size = matrix.size();
  float v_mNorm = fabs(v_m[0]);

  for (int i = 1; i < size; ++i) {
    if (v_mNorm < fabs(v_m[i])) {
      v_mNorm = fabs(v_m[i]); // вычисление нормы вектора v^m
    }
  }

  for (int i = 1; i < size; ++i) {
    if (fabs(v_m[maxAbsDenIndex] - lambda1 * u_mMinus[maxAbsDenIndex]) <
        fabs(v_m[i] - lambda1 * u_mMinus[i])) {
      maxAbsDenIndex = i; // нахождение индекса i, для которого v^m_i - lambda_1 * u^{m - 1}_i 
                            // равно максимум норме вектора v^m - lambda_1 * u^{m - 1} 
    }
  }

  return (v_mPlus[maxAbsDenIndex] * v_mNorm - lambda1 * v_m[maxAbsDenIndex])
      / (v_m[maxAbsDenIndex] - lambda1 * u_mMinus[maxAbsDenIndex]);
}

std::vector<float> PowerMethod::getSecondLambdaEigenVector(std::vector<float> v_mPlus,
                                               std::vector<float> u_m,
                                               float lambda1) {
  int size = matrix.size();
  std::vector<float> x(size);

  for (int i = 0; i < size; ++i) {
    x[i] = v_mPlus[i] - lambda1 * u_m[i];
  }

  return x;
}

std::vector<float> PowerMethod::getSecondLambdaErrorVector(std::vector<float> x, float lambda2) {
  int size = matrix.size();

  std::vector<float> error(size);

  for (int i = 0; i < size; ++i) {
    error[i] = 0;

    for (int j = 0; j < size; ++j) {
      error[i] = matrix[i][j] * x[j];
    }

    error[i] -= lambda2 * x[i];
  }

  return error;
}
```

main.cpp

```c++
const int N = 12;

int main() {
  Matrix m(N);

  std::vector<float> u_m30Minus; // значение вектора u^{m - 1} для m = 30 для вычисления lambda_1
  std::vector<float> v_m30; // значение вектора v^m для m = 30 для вычисления lambda_1 
  std::vector<float> v_m30Plus; // значение вектора v^{m + 1} для m = 30 для вычисления lambda_1 

  std::vector<float> u_m50Minus; // значение вектора u^{m - 1} для m = 50 для вычисления lambda_1 
  std::vector<float> v_m50; // значение вектора v^m для m = 50 для вычисления lambda_1 
  std::vector<float> v_m50Plus; // значение вектора v^{m + 1} для m = 50 для вычисления lambda_1 

  std::vector<float> u_k; // значение вектора u^k при k = 50 
  std::vector<float> v_kPlus; // значение вектора v^{k + 1} при k = 50 

  PrintToFile::printMatrix(m, "matrix"); // вывод матрицы в файл "matrix.txt"

  PowerMethod powerMethod(m); 
  
  for (int i = 0; i < 46; ++i) { // первые 45 итераций метода
    if (i == 30) {
      u_m30Minus = powerMethod.getU();
      v_m30 = powerMethod.getV();

      powerMethod.calcNextIt();

      v_m30Plus = powerMethod.getV();
    }

    powerMethod.calcNextIt();
  }

  std::vector<float> errorVector;
  float error;
  for (int i = 46; i < 51; ++i) { // итерации 46 - 50 с выводом собственного значения lambda_1, собственного вектора,
                                  // отвечающего значению lambda_1, вектора погрешности и максимум нормы
                                  // вектора погрешности в отдельный файл
    if (i == 50) {
      u_m50Minus = powerMethod.getU();
      v_m50 = powerMethod.getV();
    }

    powerMethod.calcNextIt();

    errorVector = powerMethod.getFirstLambdaErrorVector();
    error = fabs(errorVector[0]);
    for (int i = 1; i < N; ++i) {
      if (fabs(error) < fabs(errorVector[i])) {
        error = fabs(errorVector[i]);
      }
    }

    PrintToFile::printEigen(powerMethod.getFirstLambdaCurEigenVal(true),
                            powerMethod.getU(),
                            errorVector,
                            error,
                            "lambda" + std::to_string(i));
  }

  u_k = powerMethod.getU();
  v_m50Plus = powerMethod.getV();
  v_kPlus = powerMethod.getV();

  float lambda1, lambda2, lambda2Error;
  std::vector<float> x;
  std::vector<float> lambda2ErrorVector;

  // вычисление lambda_2 и сопутствующих значений при m = 30 и 
  // при вычислении lambda_1 при помощи формулы для несимметричнх матриц

  lambda1 = powerMethod.getFirstLambdaCurEigenVal(false);
  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m30Minus, v_m30, v_m30Plus, lambda1);
  x = powerMethod.getSecondLambdaEigenVector(v_kPlus, u_k, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2FirstCase");

  // вычисление lambda_2 и сопутствующих значений при m = 50 и 
  // при вычислении lambda_1 при помощи формулы для несимметричнх матриц

  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m50Minus, v_m50, v_m50Plus, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2SecondCase");

  // вычисление lambda_2 и сопутствующих значений при m = 50 и 
  // при вычислении lambda_1 при помощи формулы для симметричнх матриц

  lambda1 = powerMethod.getFirstLambdaCurEigenVal(true);
  lambda2 = powerMethod
      .getSecondLambdaEigenVal(u_m50Minus, v_m50, v_m50Plus, lambda1);
  x = powerMethod.getSecondLambdaEigenVector(v_kPlus, u_k, lambda1);
  lambda2ErrorVector = powerMethod.getSecondLambdaErrorVector(x, lambda2);

  lambda2Error = fabs(lambda2ErrorVector[0]);
  for (int i = 1; i < N; ++i) {
    if (fabs(lambda2Error) < fabs(lambda2ErrorVector[i])) {
      lambda2Error = fabs(lambda2ErrorVector[i]);
    }
  }

  PrintToFile::printEigen(lambda2, x, lambda2ErrorVector,
                          lambda2Error, "lambda2ThirdCase");

  return 0;
}
```

### Выходные данные

$$
\begin{array}{cccccc}
    k & 46 & 47 & 48 & 49 & 50 \\
    \lambda_1^{sym} & 33.8382607 & 33.8404007 & 33.8423233 & 33.8440628 & 33.8456154 \\
    X^{sym} & \left( \begin{array}{r}
    0.4151469 \\
    0.2251789 \\
    -0.2883672 \\
    0.1443021 \\
    0.1907129 \\
    -1.0000000 \\
    0.0711516 \\
    -0.0550737 \\
    0.9452692 \\
    0.0035368 \\
    0.0290396 \\
    -0.6685378
\end{array} \right) & \left( \begin{array}{r}
    0.4133045 \\
    0.2269261 \\
    -0.2913115 \\
    0.1393235 \\
    0.1884876 \\
    -1.0000000 \\
    0.0723689 \\
    -0.0549231 \\
    0.9528183 \\
    0.0049672 \\
    0.0292670 \\
    -0.6689291
\end{array} \right) & \left( \begin{array}{r}
    0.4115674 \\
    0.2285806 \\
    -0.2941141 \\
    0.1345730 \\
    0.1863680 \\
    -1.0000000 \\
    0.0735351 \\
    -0.0547785 \\
    0.9600456 \\
    0.0062929 \\
    0.0294957 \\
    -0.6693224
\end{array} \right) & \left( \begin{array}{r}
    0.4099288 \\
    0.2301474 \\
    -0.2967816 \\
    0.1300417 \\
    0.1843492 \\
    -1.0000000 \\
    0.0746520 \\
    -0.0546397 \\
    0.9669620 \\
    0.0075216 \\
    0.0297247 \\
    -0.6697161
\end{array} \right) & \left( \begin{array}{r}
    0.4083828 \\
    0.2316313 \\
    -0.2993205 \\
    0.1257212 \\
    0.1824268 \\
    -1.0000000 \\
    0.0757214 \\
    -0.0545067 \\
    0.9735788 \\
    0.0086610 \\
    0.0299529 \\
    -0.6701089
\end{array} \right) \\ 
\nu^{k+1} – \lambda_1^{sym} u^k & \left( \begin{array}{r}
    -0.0974255 \\
    0.0398593 \\
    -0.0749016 \\
    -0.1802940 \\
    -0.0913000 \\
    0.0848846 \\
    0.0350485 \\
    0.0097582 \\
    0.1745682 \\
    0.0479821 \\
    0.0052125 \\
    0.0435410
\end{array} \right) & \left( \begin{array}{r}
    -0.0926809 \\
    0.0371647 \\
    -0.0706186 \\
    -0.1718426 \\
    -0.0870786 \\
    0.0823555 \\
    0.0334094 \\
    0.0094060 \\
    0.1655083 \\
    0.0443413 \\
    0.0053111 \\
    0.0418129
\end{array} \right) & \left( \begin{array}{r}
    -0.0881577 \\
    0.0346627 \\
    -0.0665989 \\
    -0.1637220 \\
    -0.0830283 \\
    0.0797806 \\
    0.0318432 \\
    0.0090554 \\
    0.1569214 \\
    0.0409843 \\
    0.0053765 \\
    0.0401077
\end{array} \right) & \left( \begin{array}{r}
    -0.0838518 \\
    0.0323381 \\
    -0.0628176 \\
    -0.1559286 \\
    -0.0791454 \\
    0.0772018 \\
    0.0303462 \\
    0.0087092 \\
    0.1487770 \\
    0.0378904 \\
    0.0054122 \\
    0.0384426
\end{array} \right) & \left( \begin{array}{r}
    -0.0797491 \\
    0.0301809 \\
    -0.0592718 \\
    -0.1484561 \\
    -0.0754209 \\
    0.0746002 \\
    0.0289159 \\
    0.0083661 \\
    0.1410675 \\
    0.0350367 \\
    0.0054212 \\
    0.0368023
\end{array} \right) \\ 
\| \nu^{k+1} – \lambda_1^{sym} u^k \|_{\infty} & 0.1802940 & 0.1718426 & 0.1637220 & 0.1559286 & 0.1484561 \\
\lambda_1^{not sym} & 33.7533760 & 33.7580452 & 33.7625427 & 33.7668610 & 33.7710152 \\
    X^{not sym} & \left( \begin{array}{r}
    0.4151469 \\
    0.2251789 \\
    -0.2883672 \\
    0.1443021 \\
    0.1907129 \\
    -1.0000000 \\
    0.0711516 \\
    -0.0550737 \\
    0.9452692 \\
    0.0035368 \\
    0.0290396 \\
    -0.6685378
\end{array} \right) & \left( \begin{array}{r}
    0.4133045 \\
    0.2269261 \\
    -0.2913115 \\
    0.1393235 \\
    0.1884876 \\
    -1.0000000 \\
    0.0723689 \\
    -0.0549231 \\
    0.9528183 \\
    0.0049672 \\
    0.0292670 \\
    -0.6689291
\end{array} \right) & \left( \begin{array}{r}
    0.4115674 \\
    0.2285806 \\
    -0.2941141 \\
    0.1345730 \\
    0.1863680 \\
    -1.0000000 \\
    0.0735351 \\
    -0.0547785 \\
    0.9600456 \\
    0.0062929 \\
    0.0294957 \\
    -0.6693224
\end{array} \right) & \left( \begin{array}{r}
    0.4099288 \\
    0.2301474 \\
    -0.2967816 \\
    0.1300417 \\
    0.1843492 \\
    -1.0000000 \\
    0.0746520 \\
    -0.0546397 \\
    0.9669620 \\
    0.0075216 \\
    0.0297247 \\
    -0.6697161
\end{array} \right) & \left( \begin{array}{r}
    0.4083828 \\
    0.2316313 \\
    -0.2993205 \\
    0.1257212 \\
    0.1824268 \\
    -1.0000000 \\
    0.0757214 \\
    -0.0545067 \\
    0.9735788 \\
    0.0086610 \\
    0.0299529 \\
    -0.6701089
\end{array} \right) \\ 
\nu^{k+1} – \lambda_1^{not sym} u^k & \left( \begin{array}{r}
    -0.0621853 \\
    0.0589738 \\
    -0.0993795 \\
    -0.1680450 \\
    -0.0751114 \\
    0.0000000 \\
    0.0410883 \\
    0.0050833 \\
    0.2548084 \\
    0.0482823 \\
    0.0076775 \\
    -0.0132084
\end{array} \right) & \left( \begin{array}{r}
    -0.0586433 \\
    0.0558534 \\
    -0.0946102 \\
    -0.1603684 \\
    -0.0715556 \\
    0.0000000 \\
    0.0393693 \\
    0.0048828 \\
    0.2439804 \\
    0.0447503 \\
    0.0077214 \\
    -0.0132771
\end{array} \right) & \left( \begin{array}{r}
    -0.0553226 \\
    0.0528989 \\
    -0.0900631 \\
    -0.1529860 \\
    -0.0681601 \\
    0.0000000 \\
    0.0377097 \\
    0.0046850 \\
    0.2335129 \\
    0.0414864 \\
    0.0077296 \\
    -0.0132923
\end{array} \right) & \left( \begin{array}{r}
    -0.0522041 \\
    0.0501056 \\
    -0.0857296 \\
    -0.1458888 \\
    -0.0649133 \\
    0.0000000 \\
    0.0361094 \\
    0.0044909 \\
    0.2234268 \\
    0.0384711 \\
    0.0077070 \\
    -0.0132599
\end{array} \right) & \left( \begin{array}{r}
    -0.0492840 \\
    0.0474606 \\
    -0.0816011 \\
    -0.1390772 \\
    -0.0618119 \\
    0.0000000 \\
    0.0345647 \\
    0.0042999 \\
    0.2136955 \\
    0.0356828 \\
    0.0076556 \\
    -0.0131874
\end{array} \right) \\ 
\| \nu^{k+1} – \lambda_1^{not sym} u^k \|_{\infty} & 0.2548084 & 0.2439804 & 0.2335129 & 0.2234268 & 0.2136955
\end{array} 
$$

$$
\begin{array}{cccc}
    m & 30 & 50 & 50 \\
    \small \text{formulae for} & \text{not symmetric} & \text{not symmetric} & \text{symmetric} \\
    \lambda_2 & 32.5244179 & 32.8873901 & 32.1070137 \\ 
    X &
    \left( \begin{array}{r}
        -0.0492840 \\
        0.0474606 \\
        -0.0816011 \\
        -0.1390772 \\
        -0.0618119 \\
        0.0000000 \\
        0.0345647 \\
        0.0042999 \\
        0.2136955 \\
        0.0356828 \\
        0.0076556 \\
        -0.0131874
    \end{array} \right) &
    \left( \begin{array}{r}
        -0.0492840 \\
        0.0474606 \\
        -0.0816011 \\
        -0.1390772 \\
        -0.0618119 \\
        0.0000000 \\
        0.0345647 \\
        0.0042999 \\
        0.2136955 \\
        0.0356828 \\
        0.0076556 \\
        -0.0131874
    \end{array} \right) &
    \left( \begin{array}{r}
        -0.0797491 \\
        0.0301809 \\
        -0.0592718 \\
        -0.1484561 \\
        -0.0754209 \\
        0.0746002 \\
        0.0289159 \\
        0.0083661 \\
        0.1410675 \\
        0.0350367 \\
        0.0054212 \\
        0.0368023
    \end{array} \right) \\
    A x – \lambda_2 x &
    \left( \begin{array}{r}
        1.6424950 \\
        -1.5040647 \\
        2.6672170 \\
        4.5761542 \\
        2.0367718 \\
        -0.0000000 \\
        -1.0846356 \\
        -0.1134762 \\
        -6.8975730 \\
        -1.1605632 \\
        -0.2226198 \\
        0.1124150
    \end{array} \right) &
    \left( \begin{array}{r}
        1.6603837 \\
        -1.5212916 \\
        2.6968360 \\
        4.6266356 \\
        2.0592077 \\
        -0.0000000 \\
        -1.0971817 \\
        -0.1150370 \\
        -6.9751387 \\
        -1.1735151 \\
        -0.2253986 \\
        0.1172017
    \end{array} \right) &
    \left( \begin{array}{r}
        2.4500988 \\
        -1.0794265 \\
        1.8662386 \\
        4.6192727 \\
        2.3479340 \\
        -2.3951902 \\
        -1.0388095 \\
        -0.3422153 \\
        -4.6764655 \\
        -1.1249251 \\
        -0.2476619 \\
        -0.2983567
    \end{array} \right) \\
    \| A x – \lambda_2 x \|_{\infty} & 6.8975730 & 6.9751387 & 4.6764655 
\end{array}
$$

### Выводы

С увеличением числа итераций увеличивается точность вычисления наибольшего по модулю собмтвенного значения матрицы. Собственные значения сходятся быстрее на симметричных матрицах при применении формул для симметричных матриц, так как они учитывают свойства симметричных матриц.